                Data Engineering for "percentage by state -> deaths and incidents count"

In [206]:
# Dependancies
import pandas as pd
import os
import csv

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

# Import modules to declare columns and column data types
from sqlalchemy import Column, Integer, String, Float

# Import and establish Base for which classes will be constructed 
from sqlalchemy.ext.declarative import declarative_base

# Define Declarative Base 
Base = declarative_base()

In [188]:
# Use this to delete tables
# engine.execute('DROP TABLE cancer_trend_table')
# engine.execute('DROP TABLE states_percentage_table')
# Clear the metadata object, too
# Base.metadata.clear()
# Inspecting the Table to make sure the table is there
# inspector = inspect(engine)
# inspector.get_table_names()

[]

In [189]:
# Read in CSV File
df=pd.read_csv('db/percentage_by_state.csv', dtype={'Zip': 'str'})
df.head(2)

,state,abr,lat,lng,CancerType,incidence,population,percentage_incident,prevalence,alive,year,death_count,prevalence_1_year,percentage_deaths,diff
0,Alabama,AL,32.601011,-86.680736,Female Breast,18227,12434320,0.146586,"0,15",1865148,2011-2015,3394,"0,150272954",0.027295,0
1,Alaska,AK,61.302501,-158.775020,Female Breast,2100,1751388,0.119905,"0,12","210166,56",2011-2015,310,"0,120177254",0.017700,0


In [190]:
# Inspecting the Data
df.shape

(50, 15)

In [191]:
# Check columns for missing data
df.isnull().sum()

state                  0
abr                    0
lat                    0
lng                    0
CancerType             0
incidence              0
population             0
percentage_incident    0
prevalence             0
alive                  0
year                   0
death_count            0
prevalence_1_year      0
percentage_deaths      0
diff                   0
dtype: int64

In [192]:
# Get min and max (for leaflet part)
print(df['percentage_deaths'].min())
print(df['percentage_deaths'].max())

0.017700247
0.031030538


In [193]:
# Drop unnessesary columns. NOTE: data has no Missing/NaN values
dropped_df = df.drop(['CancerType', 'prevalence', 'alive', 'year', 'prevalence_1_year', 'diff'], axis=1)
dropped_df.head(2)

,state,abr,lat,lng,incidence,population,percentage_incident,death_count,percentage_deaths
0,Alabama,AL,32.601011,-86.680736,18227,12434320,0.146586,3394,0.027295
1,Alaska,AK,61.302501,-158.775020,2100,1751388,0.119905,310,0.017700


In [224]:
# Creating an engine, sqlite database and connection to the engine
engine = create_engine("sqlite:///breast_cancer.sqlite", echo=False)
conn = engine.connect()

In [195]:
# Creating a Class & table 
class Percentage(Base):
    __tablename__ = 'states_percentage_table'
    id = Column(Integer, primary_key=True)
    state = Column(String(255))
    abr = Column(String(255))
    lat = Column(Float)
    lng = Column(Float)
    incidence = Column(Float)
    population = Column(Float)
    percentage_incident = Column(Float)
    death_count = Column(Float)
    percentage_deaths = Column(Float)   

In [196]:
Base.metadata.create_all(conn)

In [197]:
# Transferring the data from dataframe to the sqlite db
dropped_df.to_sql(name='states_percentage_table', con=conn, if_exists='append', index=False)

In [198]:
# Inspecting the Table to make sure the table is there
inspector = inspect(engine)
inspector.get_table_names()

['states_percentage_table']

In [199]:
# Checking the data 
engine.execute('SELECT * FROM states_percentage_table LIMIT 20').fetchall()

[(1, 'Alabama', 'AL', 32.601011199999995, -86.6807365, 18227.0, 12434320.0, 0.14658622300000002, 3394.0, 0.027295421),
 (2, 'Alaska', 'AK', 61.3025006, -158.7750198, 2100.0, 1751388.0, 0.11990489800000001, 310.0, 0.017700247),
 (3, 'Arizona', 'AZ', 34.1682185, -111.930907, 22214.0, 16686987.0, 0.133121695, 3957.0, 0.023713089),
 (4, 'Arkansas', 'AR', 34.7519275, -92.13137840000002, 10399.0, 7527040.0, 0.138155238, 2050.0, 0.027235142),
 (5, 'California', 'CA', 37.2718745, -119.2704153, 128452.0, 96473279.0, 0.13314775, 22033.0, 0.022838447999999997),
 (6, 'Colorado', 'CO', 38.9979339, -105.55056699999999, 17933.0, 13143731.0, 0.136437668, 2796.0, 0.021272499),
 (7, 'Connecticut', 'CT', 41.518783500000005, -72.75750699999999, 16000.0, 9201326.0, 0.17388798100000002, 2288.0, 0.024865981000000002),
 (8, 'Delaware', 'DE', 39.145251, -75.41892059999999, 3994.0, 2387966.0, 0.16725531300000002, 672.0, 0.028141104),
 (9, 'District of C', 'DC', 38.899348700000004, -77.0145666, 2350.0, 1700582.0

In [200]:
# Data Engineering for d3 Chart
# Read in CSV File
df_d3=pd.read_csv('db/usa_rates.csv', dtype={'Zip': 'str'})
df_d3.head(2)

,year,incidents,deaths
0,1990,1625524.87,43500
1,1991,1697072.08,43800


In [201]:
# Creating a Class & table 
class Trend(Base):
    __tablename__ = 'cancer_trend_table'
    id = Column(Integer, primary_key=True)
    year = Column(Integer)
    incidents = Column(Float)
    deaths = Column(Float)

In [202]:
Base.metadata.create_all(conn)

In [203]:
# Transferring the data from dataframe to the sqlite db
df_d3.to_sql(name='cancer_trend_table', con=conn, if_exists='append', index=False)

In [204]:
# Inspecting the Table to make sure the table is there
inspector = inspect(engine)
inspector.get_table_names()

['cancer_trend_table', 'states_percentage_table']

In [205]:
# Check the data
engine.execute('SELECT * FROM cancer_trend_table LIMIT 20').fetchall()

[(1, 1990, 1625524.87, 43500.0),
 (2, 1991, 1697072.08, 43800.0),
 (3, 1992, 1763082.42, 44000.0),
 (4, 1993, 1827096.92, 44600.0),
 (5, 1994, 1875580.7, 44900.0),
 (6, 1995, 1915769.55, 45100.0),
 (7, 1996, 1937757.11, 44700.0),
 (8, 1997, 1949758.44, 44100.0),
 (9, 1998, 1956063.33, 43900.0),
 (10, 1999, 1963266.09, 43800.0),
 (11, 2000, 1976208.22, 43800.0),
 (12, 2001, 1990702.93, 43900.0),
 (13, 2002, 2000845.49, 43900.0),
 (14, 2003, 2010417.97, 43800.0),
 (15, 2004, 2017281.06, 43500.0),
 (16, 2005, 2034279.72, 43200.0),
 (17, 2006, 2046374.24, 42900.0),
 (18, 2007, 2055840.48, 42800.0),
 (19, 2008, 2067915.64, 43100.0),
 (20, 2009, 2083667.48, 43500.0)]

In [218]:
# Load the data in 
df_raw = pd.read_csv("./db/cytology_ml.csv")

In [219]:
# Frop NaN values
df_raw.dropna(inplace=True)

In [232]:
# Drop id column and rename class to diagnosis
new_df = df_raw.drop(columns=['id'])
model_df = new_df.rename(columns={"class": "diagnosis"})
model_df

,thickness,size,shape,adhesion,single,nuclei,chromatin,nucleoli,mitosis,diagnosis
0,5,1,1,1,2,1.0,3,1,1,0
1,5,4,4,5,7,10.0,3,2,1,0
2,3,1,1,1,2,2.0,3,1,1,0
3,6,8,8,1,3,4.0,3,7,1,0
4,4,1,1,3,2,1.0,3,1,1,0
5,8,10,10,8,7,10.0,9,7,1,1
6,1,1,1,1,2,10.0,3,1,1,0
7,2,1,2,1,2,1.0,3,1,1,0
8,2,1,1,1,2,1.0,1,1,5,0
9,4,2,1,1,2,1.0,2,1,1,0


In [227]:
# Creating a Class & table 
class Model(Base):
    __tablename__ = 'model_table'
    id = Column(Integer, primary_key=True)
    thickness = Column(Float)
    size = Column(Float)
    shape = Column(Float)
    adhesion = Column(Float)
    single = Column(Float)
    nuclei = Column(Float)
    chromatin = Column(Float)
    nucleoli = Column(Float)
    mitosis = Column(Float)
    diagnosis = Column(Float)

In [228]:
Base.metadata.create_all(conn)

In [229]:
# Transfer the data
model_df.to_sql(name='model_table', con=conn, if_exists='append', index=False)

In [230]:
# Inspecting the Table to make sure the table is there
inspector = inspect(engine)
inspector.get_table_names()

['cancer_trend_table', 'model_table', 'states_percentage_table']

In [231]:
# Checking the data 
engine.execute('SELECT * FROM model_table LIMIT 20').fetchall()

[(1, 5.0, 1.0, 1.0, 1.0, 2.0, 1.0, 3.0, 1.0, 1.0, 0.0),
 (2, 5.0, 4.0, 4.0, 5.0, 7.0, 10.0, 3.0, 2.0, 1.0, 0.0),
 (3, 3.0, 1.0, 1.0, 1.0, 2.0, 2.0, 3.0, 1.0, 1.0, 0.0),
 (4, 6.0, 8.0, 8.0, 1.0, 3.0, 4.0, 3.0, 7.0, 1.0, 0.0),
 (5, 4.0, 1.0, 1.0, 3.0, 2.0, 1.0, 3.0, 1.0, 1.0, 0.0),
 (6, 8.0, 10.0, 10.0, 8.0, 7.0, 10.0, 9.0, 7.0, 1.0, 1.0),
 (7, 1.0, 1.0, 1.0, 1.0, 2.0, 10.0, 3.0, 1.0, 1.0, 0.0),
 (8, 2.0, 1.0, 2.0, 1.0, 2.0, 1.0, 3.0, 1.0, 1.0, 0.0),
 (9, 2.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 5.0, 0.0),
 (10, 4.0, 2.0, 1.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0, 0.0),
 (11, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 3.0, 1.0, 1.0, 0.0),
 (12, 2.0, 1.0, 1.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0, 0.0),
 (13, 5.0, 3.0, 3.0, 3.0, 2.0, 3.0, 4.0, 4.0, 1.0, 1.0),
 (14, 1.0, 1.0, 1.0, 1.0, 2.0, 3.0, 3.0, 1.0, 1.0, 0.0),
 (15, 8.0, 7.0, 5.0, 10.0, 7.0, 9.0, 5.0, 5.0, 4.0, 1.0),
 (16, 7.0, 4.0, 6.0, 4.0, 6.0, 1.0, 4.0, 3.0, 1.0, 1.0),
 (17, 4.0, 1.0, 1.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0, 0.0),
 (18, 4.0, 1.0, 1.0, 1.0, 2.0, 1.0